1.連drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


2.取得BERT-base Chinese預訓練模型  解壓縮後資料夾放到GDRIVE對應目錄      
https://storage.googleapis.com/bert_models/2018_11_03/chinese_L-12_H-768_A-12.zip

3.安裝Kashgari套件 安裝完要 **restart runtine** ，restart後直接接第4點繼續跑程式

In [0]:
!pip install Kashgari

4.安裝 bert4keras 安裝完不用restart

In [0]:
!pip install git+https://www.github.com/bojone/bert4keras.git

5.切換到目錄到工作目錄 裡面要有 chinese_L-12_H-768_A-12 資料夾

In [3]:
cd drive/My Drive/1212

/content/drive/My Drive/1212


5.建立模型(NER)

In [4]:
import kashgari 
from kashgari.embeddings import BERTEmbedding
from kashgari.tasks.labeling import BiGRU_Model
from kashgari.corpus import ChineseDailyNerCorpus


bert_embedding = BERTEmbedding('chinese_L-12_H-768_A-12', 
                               task=kashgari.LABELING,
                               sequence_length=30)
model = BiGRU_Model(bert_embedding)


6.有要重新訓練的話跑這三行  不訓練可以下載我訓練的model放到工作目錄進行調用   
我自己訓練完的model   
https://drive.google.com/open?id=1ahwAnOQG8zu3k5WfkieM0S-s4yx1nSqV   
(裡面兩個檔案要放在BERT_ChineseDailyNerCorpus_model名稱的資料夾下)   
(BERT_ChineseDailyNerCorpus_model 資料夾要放在工作目錄   )

In [18]:
ls

BERT_ChineseDailyNerCorpus_model/  chinese_L-12_H-768_A-12/


In [0]:
# train_x, train_y = ChineseDailyNerCorpus.load_data()
# model.fit(train_x, train_y)
# model.save('BERT_ChineseDailyNerCorpus_model') 

7.調用模型(NER)

In [5]:
 loaded_model =  kashgari.utils.load_model('BERT_ChineseDailyNerCorpus_model')

8.bert4keras套件 & 加載maskLM模型

In [0]:
from bert4keras.bert import build_bert_model
from bert4keras.tokenizer import Tokenizer
import numpy as np


config_path = './chinese_L-12_H-768_A-12/bert_config.json'
checkpoint_path = './chinese_L-12_H-768_A-12/bert_model.ckpt'
dict_path = './chinese_L-12_H-768_A-12/vocab.txt'

tokenizer = Tokenizer(dict_path) # 建立分词器
model = build_bert_model(config_path, checkpoint_path, with_mlm=True) # 建立模型，加载权重

程式部分

In [0]:
def correction():
  print('請輸入文字')
  n = input()
  list1 = list(n[:29])
  list2=[]
  list2.append(list1)
  i , j , k  = 0 , 29 ,len(n)


  while( k >= 30):
     list2.append(list(n[j : j + 29 ]))
     j = j + 29
     k = k - 29

  list3 = []
  list3 = loaded_model.predict(list2)
  posList = []

#預測NER label
  count = 0
  for num_1 , i in enumerate(list3):
    for num_2 , j in enumerate(i):
      if(j != 'O'):
        posList.append(count)
      count = count + 1

# 預測mask
  result = ''

  for i in range(1,len(n)+1):
    token_ids, segment_ids = tokenizer.encode(n,'utf-8')
    if i in posList or not n[i-1].isalpha():
        result = result + n[i-1]
        continue
    token_ids[i] = tokenizer._token_dict['[MASK]']
    probas = model.predict([np.array([token_ids]), np.array([segment_ids])])[0]
    x = probas[i:i+1].argsort()[0][-5:]
    de = tokenizer.decode(x)
    if(n[i-1] not in de):
      result = result +' \033[1;34m ( \033[0m'+n[i-1]+'>'+ tokenizer.decode(probas[i:i+1].argmax(axis=1))+'\033[1;34m ) \033[0m'
    else:
      result = result + n[i-1]   
  return result 

In [14]:
print(correction())

請輸入文字
段譽—大理國「鎮南王」段正淳的世子，涉足江湖後和喬峰(蕭峰)、虛竹義結金蘭，排行三弟。
段譽—大理國「鎮南王」段正  ( 淳>義 ) 的  ( 世>兒 ) 子，涉足江湖後和喬峰(蕭峰)、虛  ( 竹>雲 ) 義結金蘭，排行三弟。


In [15]:
print(correction())

請輸入文字
宋理宗景定元年，第三次華山論劍後，郭靖之女郭襄一人騎著青驢，走遍天下，尋訪楊過和小龍女。三年後，郭襄來到河南少林寺，尋找楊過的好友、羅漢堂首座無色禪師詢問楊、龍二人的下落不果。
宋理宗景  ( 定>德 ) 元年，第三次華山論劍後，郭靖之女郭襄一人騎著  ( 青>白 )   ( 驢>龍 ) ，走遍天下，尋訪楊過和小龍女。三年後，郭襄來到河南少林寺，尋找楊過的好友、羅漢  ( 堂>山 ) 首座  ( 無>妙 )   ( 色>量 ) 禪師詢問楊、  ( 龍>郭 ) 二人的下落不果。


In [16]:
print(correction())

請輸入文字
武當派弟子「銀鉤鐵劃」張翠山前往江南調查俞岱巖受襲之事，卻發現受託運送受傷的俞岱巖的龍門鏢局慘遭滅門，又認識了天鷹教教主殷天正女兒，紫微堂堂主殷素素，更得知龍門鏢局滅門慘案是殷素素所為。
武當派弟子「銀鉤  ( 鐵>計 )   ( 劃>爪 ) 」張翠  ( 山>雲 ) 前往江南調查俞岱巖受  ( 襲>傷 ) 之事，卻發現受  ( 託>命 ) 運  ( 送>而 ) 受傷的俞岱巖的龍門鏢局慘遭滅門，  ( 又>並 ) 認識了天鷹教教主殷天  ( 正>龍 ) 女兒，紫微堂堂主殷素素，更得知龍門鏢局滅門慘案是殷素素所為。


# 未來方向&可能方向   
Fine tuning模型   
加入混淆詞   
拼音   
辨識錯詞   
結巴切詞   
專有名詞的錯誤識別   
自己建立模型、疊隱藏層   

# 參考
基于深度学习的中文文本自动校对研究与实现   
https://github.com/shibing624/pycorrector/blob/master/docs/%E5%9F%BA%E4%BA%8E%E6%B7%B1%E5%BA%A6%E5%AD%A6%E4%B9%A0%E7%9A%84%E4%B8%AD%E6%96%87%E6%96%87%E6%9C%AC%E8%87%AA%E5%8A%A8%E6%A0%A1%E5%AF%B9%E7%A0%94%E7%A9%B6%E4%B8%8E%E5%AE%9E%E7%8E%B0.pdf
